### LS/DR9 Photometry for DESI/EDR

This notebook briefly shows how to work with the LS/DR9 value-added catalogs for a small set of targets in the DESI Early Data Release (EDR).

John Moustakas  
Siena College  
2022 June XX

In [2]:
import os
import numpy as np
import fitsio
from glob import glob
from astropy.table import Table, vstack

In [4]:
from desimodel.footprint import radec2pix
from desitarget.sv3.sv3_targetmask import desi_mask

#### Specify the location of the DESI/EDR redshift catalogs and the LS/DR9 VAC.

In [3]:
zcatdir = '/global/cfs/cdirs/desi/public/edr/spectro/redux/fuji/zcatalog'
vacdir = '/global/cscratch1/sd/ioannis/lsdr9-photometry/v1.0'

#### Select a small number of LRG targets.

In [5]:
zcat = Table(fitsio.read(os.path.join(zcatdir, 'ztile-sv3-dark-cumulative.fits'), 'ZCATALOG'))
I = (zcat['SV3_DESI_TARGET'] & desi_mask.mask('LRG') != 0) * (zcat['ZWARN'] == 0) * (zcat['Z'] > 0.7) * (zcat['Z'] < 0.701)
zcat = zcat[I]
print('Selected {} objects'.format(len(zcat)))

OSError: FITSIO status = 104: could not open the named file
failed to find or open the following file: (ffopen)
/global/cfs/cdirs/desi/public/edr/spectro/redux/fuji/zcatalog/ztile-sv3-dark-cum
ulative.fits


As an example, to grab the photometry of your favorite, say, LRG targets observed in SV3, you would do something like the following (which we should of course put into a notebook):

In [5]:
import healpy as hp
hp.nside2resol(4, arcmin=True) / 60

14.658075357087599

In [ ]:
def get_tractorphot():
    from desimodel.footprint import radec2pix
    from desitarget.sv3.sv3_targetmask import desi_mask

    zcat = Table(fitsio.read(os.path.join(zcatdir, 'ztile-sv3-dark-cumulative.fits'), 'ZCATALOG'))
    I = (zcat['SV3_DESI_TARGET'] & desi_mask.mask('LRG') != 0) * (zcat['ZWARN'] == 0) * (zcat['Z'] > 0.7) * (zcat['Z'] < 0.701)
    zcat = zcat[I]
    print('Selected {} objects'.format(len(zcat)))

    hdr = fitsio.read_header(glob(os.path.join(photdir, 'tractorphot-nside4-hp???-fuji.fits'))[0], 'TRACTORPHOT')
    tractorphot_nside = hdr['FILENSID']

    pixels = radec2pix(tractorphot_nside, zcat['TARGET_RA'], zcat['TARGET_DEC'])
    phot = []
    for pixel in set(pixels):
        J = pixel == pixels
        photfile = os.path.join(photdir, 'tractorphot-nside4-hp{:03d}-fuji.fits'.format(pixel))
        targetids = fitsio.read(photfile, columns='TARGETID')
        K = np.where(np.isin(targetids, zcat['TARGETID'][J]))[0]
        print('Reading photometry for {} objects from {}'.format(len(K), photfile))
        _phot = fitsio.read(photfile, rows=K)
        phot.append(Table(_phot))
    phot = vstack(phot)
    srt = np.hstack([np.where(tid == phot['TARGETID'])[0] for tid in zcat['TARGETID']]) # better way to sort here??
    phot = phot[srt]
    return zcat, phot

>>> zcat, phot = get_tractorphot()